## In this notebook, we will load and clean our data
 We are working with 10 files, but df_train_all_v1.parquet is the full training data set we can start with



In [8]:
# Core fact tables (sales data):
#	•	df2016_train.parquet (subset, probably for initial testing/competition baseline)
#	•	df_train_all_v1.parquet (full training set)
#	•	df_test.parquet, df_test_2022.parquet (test sets)
#	•	sample_submission.parquet (submission format, not for analysis)

# Supporting dimension tables (metadata & signals):
#	•	items.parquet (item family, perishable flag, etc.)
#	•	stores.parquet (store city, type, cluster)
#	•	transactions.parquet (daily transaction counts by store)
#	•	holiday_events.parquet (dates + holiday flags)
#	•	oil.parquet (daily oil prices)

In [9]:
#add libraries to use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
import os
print(os.getcwd())

/Users/angiediaz/CPSC4300/grocery-sales-forecasting/scripts


In [11]:
# Load in df_train_all_v1.parquet (main) + rest 
df_train = pd.read_parquet("../data/raw/df_train_all_v1.parquet")
df_items = pd.read_parquet("../data/raw/items.parquet")
df_stores = pd.read_parquet("../data/raw/stores.parquet")

df_holidays = pd.read_parquet("../data/raw/holiday_events.parquet")
df_oil = pd.read_parquet("../data/raw/oil.parquet")
df_transactions = pd.read_parquet("../data/raw/transactions.parquet")


#Extra datasets, not used for modeling just checking these are also good
df_test = pd.read_parquet("../data/raw/df_test.parquet")
df_test_2022 = pd.read_parquet("../data/raw/df_test_2022.parquet")
df_sample_submission = pd.read_parquet("../data/raw/sample_submission.parquet")

In [12]:
#Check its shape, types and missing values

print(df_train.shape)
print(df_train.dtypes)
print(df_train.isnull().sum())

# Check duplicates
dup_count = df_train.duplicated().sum()
print(f"\nNumber of duplicate rows: {dup_count}")

df_train.info()

(125497040, 5)
date           datetime64[us]
store_nbr               int64
item_nbr                int64
unit_sales            float64
onpromotion              bool
dtype: object
date           0
store_nbr      0
item_nbr       0
unit_sales     0
onpromotion    0
dtype: int64

Number of duplicate rows: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   date         datetime64[us]
 1   store_nbr    int64         
 2   item_nbr     int64         
 3   unit_sales   float64       
 4   onpromotion  bool          
dtypes: bool(1), datetime64[us](1), float64(1), int64(2)
memory usage: 3.9 GB


In [13]:
df_train.columns

Index(['date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion'], dtype='object')

In [14]:
df_train[df_train['store_nbr']==1]
#not organized by str_nbr, but by date

,date,store_nbr,item_nbr,unit_sales,onpromotion
578,2013-01-02,1,103665,1.098612,False
579,2013-01-02,1,105574,2.197225,False
580,2013-01-02,1,105575,2.772589,False
581,2013-01-02,1,105577,1.098612,False
582,2013-01-02,1,105737,1.098612,False
...,...,...,...,...,...
125396207,2017-08-15,1,2114812,0.693147,True
125396208,2017-08-15,1,2116416,0.693147,False
125396209,2017-08-15,1,2122188,0.693147,False
125396210,2017-08-15,1,2122676,1.098612,False


In [15]:
df_train[df_train['unit_sales']<0]

,date,store_nbr,item_nbr,unit_sales,onpromotion


In [16]:
# So after looking at the data, we can see that there are no missing values in the dataset
# We can also see that the date column is already in datetime format
# All the other columns have the correct type
# This is a huge dataset, will need to be cautious with memory usage

In [17]:
# Check the other datasets for missing values and types

def check_df(df, name):
    print(f"Checking {name}")
    print(df.shape)
    print(df.dtypes)
    print(df.isnull().sum())
    print(df.info())
    
    # Check duplicates
    dup_count = df.duplicated().sum()
    print(f"\nNumber of duplicate rows: {dup_count}")
    
    print("\n")

In [18]:
check_df(df_items, "items")


#So in the items dataset, there are no missing values 
# the perishable column is an integer, but it should be a boolean 
# will match our onpromo column in the main dataset


df_items["perishable"] = df_items["perishable"].astype(bool)

df_items.head()

#also the range runs from 96995 --> 2134244
df_items = df_items.reset_index(drop=True)

Checking items
(4100, 3)
family        object
class          int64
perishable     int64
dtype: object
family        0
class         0
perishable    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 4100 entries, 96995 to 2134244
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   family      4100 non-null   object
 1   class       4100 non-null   int64 
 2   perishable  4100 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 128.1+ KB
None

Number of duplicate rows: 3763




In [19]:
# So we have 4100 rows, but only 410 unique item_ids
# We don't need to drop them though, only when we want a version for visualizations

In [20]:
check_df(df_stores, "stores")

df_stores.head()

#Interesting thing in this dataset: cluster column
# This is a grouping of stores based on sales and volume
# Cluster is a grouping of stores with similar characteristics, but
# the way it is calculated is not given, (could be similar sales patterns, demographics, or customer profiles)


#address this in EDA, need to see if this could be a predictive value, if so, we keep, if not we drop

#Some rows are duplicates in terms of city, state, type and cluster, but each store is uniquely identified by store_nbr
# Duplicates do not affect merges with the main dataset 

Checking stores
(54, 4)
city       object
state      object
type       object
cluster     int64
dtype: object
city       0
state      0
type       0
cluster    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 54 entries, 1 to 54
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   city     54 non-null     object
 1   state    54 non-null     object
 2   type     54 non-null     object
 3   cluster  54 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 2.1+ KB
None

Number of duplicate rows: 12




,city,state,type,cluster
store_nbr,,,,
1,Quito,Pichincha,D,13
2,Quito,Pichincha,D,13
3,Quito,Pichincha,D,8
4,Quito,Pichincha,D,9
5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [21]:
# Average and total unit sales per cluster
cluster_sales = (
    df_train
    .groupby("store_nbr")["unit_sales"].sum()
    .reset_index()
    .merge(df_stores[["cluster"]], left_on="store_nbr", right_index=True)
    .groupby("cluster")["unit_sales"]
    .agg(["mean", "sum", "count"])
    .reset_index()
)

print(cluster_sales)


# Don't see a clear pattern here, so we will keep the cluster column for now
# Stores into 17 clusters, but initial analysis shows no strong or consistent relationship 
# between cluster and sales volume (average), may still be useful in modeling 

    cluster          mean           sum  count
0         1  4.134454e+06  1.240336e+07      3
1         2  3.516715e+06  7.033429e+06      2
2         3  2.641090e+06  1.848763e+07      7
3         4  4.217771e+06  1.265331e+07      3
4         5  8.145786e+06  8.145786e+06      1
5         6  3.887535e+06  2.332521e+07      6
6         7  2.071533e+06  4.143066e+06      2
7         8  6.270882e+06  1.881265e+07      3
8         9  4.043923e+06  8.087847e+06      2
9        10  3.273356e+06  1.964013e+07      6
10       11  5.126212e+06  1.537864e+07      3
11       12  3.885254e+06  3.885254e+06      1
12       13  4.370101e+06  1.748040e+07      4
13       14  6.747228e+06  2.698891e+07      4
14       15  2.823346e+06  1.411673e+07      5
15       16  3.429654e+06  3.429654e+06      1
16       17  5.715226e+06  5.715226e+06      1


In [22]:
#Checking to see if clusters have geographical patterns
# Merge cluster info into stores table
stores_clusters = df_stores[["city", "state", "cluster"]]

# See which clusters appear in which states
cluster_state = stores_clusters.groupby("cluster")["state"].unique().reset_index()
print(cluster_state)

# Optional: count of stores per cluster per state
cluster_state_count = stores_clusters.groupby(["cluster", "state"]).size().reset_index(name="count")
print(cluster_state_count)


# Clusters do not appear to be strictly geographical, as many clusters span multiple states

    cluster                                              state
0         1                              [Guayas, Santa Elena]
1         2                                            [Azuay]
2         3  [Santo Domingo de los Tsachilas, Guayas, Los R...
3         4     [Santo Domingo de los Tsachilas, Loja, El Oro]
4         5                                        [Pichincha]
5         6  [Pichincha, Santo Domingo de los Tsachilas, Gu...
6         7                              [Chimborazo, Pastaza]
7         8                                        [Pichincha]
8         9                            [Pichincha, Tungurahua]
9        10                     [Guayas, Los Rios, Esmeraldas]
10       11                                [Pichincha, Manabi]
11       12                                        [Pichincha]
12       13                                [Pichincha, Manabi]
13       14                            [Pichincha, Tungurahua]
14       15           [Pichincha, Cotopaxi, Imbabura, B

In [23]:
check_df(df_holidays, "holidays")

df_holidays.head()


# Transferred column: indicates if the holiday is observed on a different date than the actual holiday
# Will be important when we want to flag the day customers actually get the holiday off (sales might spike on transferred days than official holiday)

Checking holidays
(350, 6)
date           object
type           object
locale         object
locale_name    object
description    object
transferred      bool
dtype: object
date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB
None

Number of duplicate rows: 0




,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [24]:
check_df(df_transactions, "transactions")

Checking transactions
(83488, 3)
date            object
store_nbr        int64
transactions     int64
dtype: object
date            0
store_nbr       0
transactions    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB
None

Number of duplicate rows: 0




In [25]:
# Currently, date is an object (string), convert to datetime object

df_transactions["date"] = pd.to_datetime(df_transactions["date"], format="%Y-%m-%d")
print(df_transactions.dtypes)

date            datetime64[ns]
store_nbr                int64
transactions             int64
dtype: object


In [26]:
#Purpose of oil dataset:
# daily closing price of crude oil
# Could be a proxy for broader economic conditions, as oil prices can influence transportation costs, consumer spending, and overall economic activity
    # e.g transportation costs, seasonal/holiday shopping patterns 

check_df(df_oil, "oil")

Checking oil
(1218, 2)
date           object
dcoilwtico    float64
dtype: object
date           0
dcoilwtico    43
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB
None

Number of duplicate rows: 0




In [27]:
# Date is also an object (string), convert to datetime object
df_oil["date"] = pd.to_datetime(df_oil["date"], format="%Y-%m-%d")

# 43 missing values in the dcoilwtico column
df_oil["dcoilwtico"] = df_oil["dcoilwtico"].interpolate(method="linear")


check_df(df_oil, "oil after cleaning")

Checking oil after cleaning
(1218, 2)
date          datetime64[ns]
dcoilwtico           float64
dtype: object
date          0
dcoilwtico    1
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1218 non-null   datetime64[ns]
 1   dcoilwtico  1217 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 19.2 KB
None

Number of duplicate rows: 0




In [28]:
df_oil[df_oil["dcoilwtico"].isnull()]

df_oil["dcoilwtico"] = df_oil["dcoilwtico"].fillna(method="bfill")

check_df(df_oil, "oil after backfill")

Checking oil after backfill
(1218, 2)
date          datetime64[ns]
dcoilwtico           float64
dtype: object
date          0
dcoilwtico    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1218 non-null   datetime64[ns]
 1   dcoilwtico  1218 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 19.2 KB
None

Number of duplicate rows: 0




/var/folders/2k/j9b6q32d02dgv4_bwwtm_lxh0000gn/T/ipykernel_90131/661833160.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_oil["dcoilwtico"] = df_oil["dcoilwtico"].fillna(method="bfill")


In [32]:
df_train.to_parquet("../data/df_train_cleaned.parquet", engine="pyarrow", index=False)
df_oil.to_parquet("../data/df_oil_cleaned.parquet", engine="pyarrow", index=False)
df_transactions.to_parquet("../data/df_train_cleaned.parquet", engine="pyarrow", index=False)
df_holidays.to_parquet("../data/df_train_cleaned.parquet", engine="pyarrow", index=False)
df_items.to_parquet("../data/df_items_cleaned.parquet", engine="pyarrow", index=False)
df_stores.to_parquet("../data/df_stores_cleaned.parquet", engine="pyarrow", index=False)
print("Cleaned datasets saved to 'data' directory.")


Cleaned datasets saved to 'data' directory.


In [33]:
# Stratified sampling to create a smaller representative dataset 

from pathlib import Path

# --- Paths ---
DATA_DIR = Path("../data")        # adjust if needed
OUTPUT_CSV = DATA_DIR / "df_train_stratified.csv"
OUTPUT_PARQUET = DATA_DIR / "df_train_stratified.parquet"

# --- Load cleaned full train dataset ---
# Assuming you saved cleaned parquet in your notebook as df_train_cleaned.parquet
df_train = pd.read_parquet(DATA_DIR / "df_train_cleaned.parquet", engine="pyarrow")

print("Full df_train shape:", df_train.shape)

# --- Stratified sample by date ---
# Keeps 1% per date, preserving time series distribution
stratified_sample = (
    df_train.groupby("date", group_keys=False)
    .apply(lambda x: x.sample(frac=0.01, random_state=42))
    .reset_index(drop=True)
)

print("Stratified sample shape:", stratified_sample.shape)

# --- Save results ---
stratified_sample.to_csv(OUTPUT_CSV, index=False)
stratified_sample.to_parquet(OUTPUT_PARQUET, engine="pyarrow", index=False)

print(f"Saved stratified sample to:\n- {OUTPUT_CSV}\n- {OUTPUT_PARQUET}")


Full df_train shape: (350, 6)
Stratified sample shape: (0, 6)
Saved stratified sample to:
- ../data/df_train_stratified.csv
- ../data/df_train_stratified.parquet


/var/folders/2k/j9b6q32d02dgv4_bwwtm_lxh0000gn/T/ipykernel_90131/2024461829.py:20: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(frac=0.01, random_state=42))


In [ ]:
check_df(df_test, "Df Test")

df_test

#organized by store number, starting with store 1, all items, all dates



Checking Df Test
(3370464, 2)
id             int64
onpromotion     bool
dtype: object
id             0
onpromotion    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3370464 entries, (np.int64(1), np.int64(96995), Timestamp('2017-08-16 00:00:00')) to (np.int64(54), np.int64(2134244), Timestamp('2017-08-31 00:00:00'))
Data columns (total 2 columns):
 #   Column       Dtype
---  ------       -----
 0   id           int64
 1   onpromotion  bool 
dtypes: bool(1), int64(1)
memory usage: 41.9 MB
None

Number of duplicate rows: 0




id  onpromotion
store_nbr item_nbr date                              
1         96995    2017-08-16  125497040        False
          99197    2017-08-16  125497041        False
          103501   2017-08-16  125497042        False
          103520   2017-08-16  125497043        False
          103665   2017-08-16  125497044        False
...                                  ...          ...
54        2132163  2017-08-31  128867499        False
          2132318  2017-08-31  128867500        False
          2132945  2017-08-31  128867501        False
          2132957  2017-08-31  128867502        False
          2134244  2017-08-31  128867503        False

[3370464 rows x 2 columns]

In [ ]:
check_df(df_test_2022, "Df Test 2022")

df_test_2022

#ok this could be useful, updated to year 2022, so we can see how sales patterns have changed post-pandemic?

Checking Df Test 2022
(3370464, 2)
id             int64
onpromotion     bool
dtype: object
id             0
onpromotion    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3370464 entries, (np.int64(1), np.int64(96995), Timestamp('2022-09-10 00:00:00')) to (np.int64(54), np.int64(2134244), Timestamp('2022-09-25 00:00:00'))
Data columns (total 2 columns):
 #   Column       Dtype
---  ------       -----
 0   id           int64
 1   onpromotion  bool 
dtypes: bool(1), int64(1)
memory usage: 41.9 MB
None

Number of duplicate rows: 0




id  onpromotion
store_nbr item_nbr date                              
1         96995    2022-09-10  125497040        False
          99197    2022-09-10  125497041        False
          103501   2022-09-10  125497042        False
          103520   2022-09-10  125497043        False
          103665   2022-09-10  125497044        False
...                                  ...          ...
54        2132163  2022-09-25  128867499        False
          2132318  2022-09-25  128867500        False
          2132945  2022-09-25  128867501        False
          2132957  2022-09-25  128867502        False
          2134244  2022-09-25  128867503        False

[3370464 rows x 2 columns]

In [ ]:
check_df(df_sample_submission, "Sample Submission")

df_sample_submission[df_sample_submission['unit_sales'] > 0]

#empty sales column, just a template for submission format

Checking Sample Submission
(3370464, 2)
id            int64
unit_sales    int64
dtype: object
id            0
unit_sales    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3370464 entries, 0 to 3370463
Data columns (total 2 columns):
 #   Column      Dtype
---  ------      -----
 0   id          int64
 1   unit_sales  int64
dtypes: int64(2)
memory usage: 51.4 MB
None

Number of duplicate rows: 0




,id,unit_sales


In [ ]:
def check_uniqueness(df, name, keys):
    total_rows = df.shape[0]
    unique_rows = df[keys].drop_duplicates().shape[0]
    
    print(f"🔎 {name}")
    print(f"  Unit of analysis: {keys}")
    print(f"  Total rows: {total_rows}")
    print(f"  Unique rows: {unique_rows}")
    if total_rows == unique_rows:
        print("  All rows are unique by this unit of analysis.\n")
    else:
        print(f"{total_rows - unique_rows} duplicate rows by this unit of analysis.\n")

In [ ]:
check_uniqueness(df_train, "Train", ["date","store_nbr","item_nbr"])
check_uniqueness(df_transactions, "Transactions", ["date","store_nbr"])
check_uniqueness(df_oil, "Oil", ["date"])
check_uniqueness(df_holidays, "Holidays", ["date","description"])
check_uniqueness(df_items, "Items", ["family","class","perishable"])
check_uniqueness(df_stores, "Stores", ["city","state","type","cluster"])

🔎 Train
  Unit of analysis: ['date', 'store_nbr', 'item_nbr']
  Total rows: 125497040
  Unique rows: 125497040
  All rows are unique by this unit of analysis.

🔎 Transactions
  Unit of analysis: ['date', 'store_nbr']
  Total rows: 83488
  Unique rows: 83488
  All rows are unique by this unit of analysis.

🔎 Oil
  Unit of analysis: ['date']
  Total rows: 1218
  Unique rows: 1218
  All rows are unique by this unit of analysis.

🔎 Holidays
  Unit of analysis: ['date', 'description']
  Total rows: 350
  Unique rows: 350
  All rows are unique by this unit of analysis.

🔎 Items
  Unit of analysis: ['family', 'class', 'perishable']
  Total rows: 4100
  Unique rows: 337
3763 duplicate rows by this unit of analysis.

🔎 Stores
  Unit of analysis: ['city', 'state', 'type', 'cluster']
  Total rows: 54
  Unique rows: 42
12 duplicate rows by this unit of analysis.

